In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

df_train = pd.read_csv('../../Deceptive_Opinion_Spam_Corpus_Datasets/deceptive_training_sin_proc.csv')
df_test = pd.read_csv('../../Deceptive_Opinion_Spam_Corpus_Datasets/deceptive_testing_sin_proc.csv')

In [2]:
### Remplazar los truthful con 0 y deceptive con 1 en la columna deceptive
df_train = df_train.replace({'deceptive': {'truthful': 0, 'deceptive': 1}})
df_test = df_test.replace({'deceptive': {'truthful': 0, 'deceptive': 1}})

In [3]:
train_dec_a = df_train.iloc[:, 0]
#validation_dec_a = validation_set.iloc[:, 0]
test_dec_a = df_test.iloc[:, 0]

train_dec_a.columns = ['train_dec_a']
#validation_dec_a.columns = ['validation_dec_a']
test_dec_a.columns = ['test_dec_a']

In [4]:
print(df_train)

     deceptive                                               text
0            0  -0.19499199 -0.11689109 -1.0239629 0.589182 -0...
1            0  -0.75981146 -0.6247913 -0.5968303 0.8442113 -0...
2            0  -0.67143595 -0.34271178 -0.6285268 0.46137583 ...
3            0  -0.3716615 -0.6101102 -0.6716857 1.4250252 -0....
4            0  0.09419579 0.02537344 -0.07499692 1.4799229 -0...
..         ...                                                ...
592          0  -0.7387875 -0.31224024 -0.71631956 0.7961637 -...
593          0  -0.6691669 -0.8395288 -0.88937396 0.82928854 -...
594          0  -0.47775698 -0.21484302 -0.7292124 0.70525247 ...
595          0  -0.051372834 -0.48699462 -0.95044464 1.4210202...
596          0  -0.71351445 0.17851919 -0.258802 1.0437325 -0....

[597 rows x 2 columns]


In [5]:

x_train = ()
y_train = ()

for index, row in df_train.iterrows():
    res = list(map(float, row['text'].split(' ')))
    res = np.asarray(res, dtype=np.float32)
    x_train = x_train + (res,)
    y_train = y_train + (row['deceptive'],)

x_test = ()
y_test = ()

for index, row in df_test.iterrows():
    res = list(map(float, row['text'].split(' ')))
    res = np.asarray(res, dtype=np.float32)
    x_test = x_test + (res,)
    y_test = y_test + (row['deceptive'],)


In [6]:
import numpy
arrays = {}
## Train dataset
for i in range(0, len(x_train[0])):
    col_name = 'value_' + str(i)
    arrays[col_name] = numpy.array([item[i] for item in x_train])

train_ext = pd.DataFrame(y_train, columns =['deceptive'])

for x in arrays.keys(): 
    dataframe=pd.DataFrame(arrays[x], columns=[x]) 
    train_ext = train_ext.join(dataframe, how="inner")
    
train_ext = train_ext.drop(['deceptive'],axis=1)
## Test dataset

for i in range(0, len(x_test[0])):
    col_name = 'value_' + str(i)
    arrays[col_name] = numpy.array([item[i] for item in x_test])

test_ext = pd.DataFrame(y_test, columns =['deceptive'])

for x in arrays.keys(): 
    dataframe=pd.DataFrame(arrays[x], columns=[x]) 
    test_ext = test_ext.join(dataframe, how="inner")
    
test_ext = test_ext.drop(['deceptive'],axis=1)


In [7]:
import numpy as np
x_train, y_train = np.asarray(x_train), np.asarray(y_train)
#x_validation, y_validation = np.asarray(x_validation), np.asarray(y_validation)
x_test, y_test = np.asarray(x_test), np.asarray(y_test)

x_train = x_train.astype('float32')
#x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')

In [8]:
# reshape inputs for LSTM [samples, timesteps, features]
X_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1]) # X_train 
print("Training data shape:", x_train.shape)
#x_validation = x_validation.reshape(x_validation.shape[0], 1, x_validation.shape[1]) # X_train 
#print("Validation data shape:", x_validation.shape)
X_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])     # X_test 
print("Test data shape:", x_test.shape)

Training data shape: (597, 300)
Test data shape: (198, 300)


In [9]:
print(X_train)

[[[-0.19499199 -0.11689109 -1.0239629  ... -0.3362849   0.09603526
   -0.29484585]]

 [[-0.75981146 -0.6247913  -0.5968303  ...  0.20598556 -0.4579486
    0.02073733]]

 [[-0.67143595 -0.34271178 -0.6285268  ... -0.49001852 -0.4839096
    0.07291023]]

 ...

 [[-0.47775698 -0.21484302 -0.7292124  ...  0.11411288  0.13427125
    0.22181219]]

 [[-0.05137283 -0.48699462 -0.95044464 ... -0.8822731  -0.40317678
    0.25194392]]

 [[-0.71351445  0.17851919 -0.258802   ...  0.2708312  -0.08303961
   -0.6212692 ]]]


## Keras Tuner

In [10]:
import tensorflow as tf
from tensorflow import keras

!pip install -q -U keras-tuner
import kerastuner as kt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
## Aparte del modelo vamos a definir los hiperparametros construyendo el hypermodelo
## Vamos a utilizar o bien una funccion builder o utilizar una subclase HyperModel usando la Keras Tuner 

from keras.layers import Dense, Input, Conv2D, Flatten, Reshape, Conv2DTranspose, LSTM, TimeDistributed, RepeatVector, Dropout
from tensorflow.keras import models, layers
from keras.models import Model, Sequential
from keras.utils import plot_model
from keras import backend as BE
from keras import regularizers
from kerastuner import HyperModel
from kerastuner import Hyperband
from kerastuner import BayesianOptimization
from kerastuner.tuners import RandomSearch

import numpy as np
import matplotlib.pyplot as plt


class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()
        model.add(
            layers.LSTM(300,
                #units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )

        L1=model.add(
            layers.LSTM(
                units=hp.Int('units_2', min_value=16, max_value=256, step=16), # 'units_2', 16, 32, 64, default=32),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                    kernel_regularizer=regularizers.l2(hp.Choice('regularizers.l2', values=[0.00, 0.01, 0.05, 0.1])), # Valorarlo... 
                return_sequences=True,
                input_shape=input_shape
            )
        ) 
        
        L2=model.add(
            layers.LSTM(
                units=hp.Int('units_3', min_value=2, max_value=32, step=2),              #'units_3', 2, 4, 8, default=4
                activation=hp.Choice( 
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )  
        
        
        L3=model.add(
            layers.LSTM(
                units=hp.Int('units_4', min_value=2, max_value=32, step=2),              #'units_3', 2, 4, 8, default=4
                activation=hp.Choice( 
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        ) 
        
        L4=model.add(
            layers.LSTM(
                units=hp.Int('units_5', min_value=16, max_value=256, step=16),        #'units_4', 16, 32, 64, default=32
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(300))
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop','nadam', 'adadelta',
                                            'adagrad', 'adamax','ftrl'])
        model.compile(
            optimizer=optimizer,loss='mse',metrics=['mse']
        )
        
        return model

In [12]:
input_shape = (X_train.shape[1],X_train.shape[2],)
hypermodel = RegressionHyperModel(input_shape)

In [13]:
## RandomSearch

In [16]:
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=10,
            max_trials= 10,
            directory='keras-tuner-deceptive',
            project_name='deceptive-opinion-keras',
            executions_per_trial=5)

In [17]:
tuner_rs.search(X_train, X_train, epochs=50, validation_split = 0.2, verbose=0)

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner_rs.search_space_summary()

In [ ]:
tuner_rs.results_summary()

In [ ]:
tuner_rs.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
model = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = model.evaluate(X_test, X_test)[1]
print('Random search MSE: ', mse_rs)

In [ ]:
## Hyperband

In [18]:
tuner_hb = Hyperband(
            hypermodel,
            max_epochs= 20,
            objective='mse',
            seed=100,
            overwrite=True,
            executions_per_trial=4
        )

In [19]:
tuner_hb.search(X_train, X_train, epochs= 100, validation_split = 0.2, verbose=0)

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner_hb.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
model = tuner_hb.get_best_models(num_models=1)[0]
mse_hb = best_model.evaluate(X_test, X_test)[1]
print('Hyperband Optimization MSE: ', mse_hb)

In [ ]:
## Bayesian Optimization

In [ ]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='mse',
            max_trials=10,
            seed=10,
            overwrite=True,
            executions_per_trial=4
        )

In [ ]:
tuner_bo.search(X_train, X_train, epochs=100, validation_split = 0.2, verbose=0) 

In [ ]:
tuner_bo.get_best_models(num_models=1)[0]
tuner_bo.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
model = tuner_bo.get_best_models(num_models=1)[0]
mse_bo = model.evaluate(x_train, x_train)[1]
print('Bayesian Optimization MSE: ', mse_bo)

In [ ]:
## Results for Keras models

In [ ]:
tuner_rs.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
tuner_rs.get_best_models()[0].summary()

In [ ]:
tuner_bo.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
tuner_bo.get_best_models()[0].summary()